# Data Wrangling on WeRateDogs Tweets
by Shan Jafri

In a world where data is abundent gathering and cleaning data is necessary for any data scientist to extract valuable insights. The focus of this project is to gather data from multiple sources, assess dirty data and then clean the data. The last step of the project is to show several insights from the clean data set.

# Gather
In this project, data was gathered from three sources:

- The WeRateDogs twitter archive that was provided by Udacity as a csv file.
- The image prediction file that was hosted by Udacity serves and downloaded programmatically using request libary
- Using twitters API and python's tweepy library, any missing information from WeRateDogs twiiter archive such as retweet counts and favorite counts are gathered. The tweet IDs in the WeRateDogs Twitter archive will be use to query the Twitter API for each JSON structure tweet using Python's Tweepy library and in a file called tweet_json.txt file. Each JSON formatted tweet should be written to its own line in a text file and then read line by line into a pandas DataFrame.

** Let's begin gathering data!**

In [105]:
# Import the libaries needed for this project
import pandas as pd 
import numpy as np 
import requests
import os
import tweepy
import json
from timeit import default_timer as timer
import re

In [20]:
# Open twitter archive enhanced and read file in pandas
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
# view the file 
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

** The file twitter archive has 17 columns and 2356 entries.**

In [21]:
# Request image prediction file from Udacity Server and read in Pandas
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

with open (url.split('/')[-1], mode = 'wb') as file:
    file.write(response.content)

image_prediction = pd.read_csv('image-predictions.tsv', sep = '\t')

In [84]:
# View image prediction file 
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


#### Extracting data using Twitters API 

In [23]:
# token and keys
consumer_key = '6GF8J1A1tZQODiu5Hch3O8Mru'
consumer_secret = 'dAQWLTIeGCYWHWCBUHaOR7QA3KVdfLaHsqky54lecigFkt2K6V'
access_token = '1047201527423815680-cnOWUtr4zWolbcFrkWabmzrvWK4BZ8'
access_token_secret = '8wMkUZH6pUQnwQU0IyX3XPwctA30yAf81tTFDZNV7Cn8P' 

# Authenticating user access
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Contruct the API instance
api = tweepy.API(auth, 
                 parser=tweepy.parsers.JSONParser(), # parse the results to JSON objects 
                 wait_on_rate_limit=True, # Automatically wait for rate limits to replenish
                 wait_on_rate_limit_notify=True) # Print notification when Tweepy is waiting to replenish

In [24]:
# Store the results
tweet_dic = []
# Store tweet error
error = []

start = timer() # Calculating the length of time
# Get tweet JSON objects for all tweets in twitter archive data set
for tweet in twitter_archive['tweet_id']:
    try:
        tweet_data = api.get_status(tweet, tweet_mode = 'extended')
        # Get JSON Data of retweet and favorite count
        retweets = tweet_data['retweet_count']
        favorites = tweet_data['favorite_count']
        
        tweet_dic.append({'tweet_id': tweet,
                          'retweet': retweets,
                          'favorite': favorites})
        
    # Catch the exceptions of the TweepError
    except Exception as e:
        print(str(tweet)+ " _ " + str(e))
        error.append(tweet)

888202515573088257 _ [{'code': 144, 'message': 'No status found with that ID.'}]
873697596434513921 _ [{'code': 144, 'message': 'No status found with that ID.'}]
872668790621863937 _ [{'code': 144, 'message': 'No status found with that ID.'}]
869988702071779329 _ [{'code': 144, 'message': 'No status found with that ID.'}]
866816280283807744 _ [{'code': 144, 'message': 'No status found with that ID.'}]
861769973181624320 _ [{'code': 144, 'message': 'No status found with that ID.'}]
845459076796616705 _ [{'code': 144, 'message': 'No status found with that ID.'}]
842892208864923648 _ [{'code': 144, 'message': 'No status found with that ID.'}]
837012587749474308 _ [{'code': 144, 'message': 'No status found with that ID.'}]
827228250799742977 _ [{'code': 144, 'message': 'No status found with that ID.'}]
802247111496568832 _ [{'code': 144, 'message': 'No status found with that ID.'}]
775096608509886464 _ [{'code': 144, 'message': 'No status found with that ID.'}]
770743923962707968 _ [{'code

Rate limit reached. Sleeping for: 598


754011816964026368 _ [{'code': 144, 'message': 'No status found with that ID.'}]


Rate limit reached. Sleeping for: 601


In [25]:
# lengh of the result
print("The lengh of the result", len(tweet_dic))
# The tweet_id of the errors
print("The lengh of the errors", len(error))

The lengh of the result 2342
The lengh of the errors 14


In [26]:
# Store tweet error
second_error = []

start = timer() # Calculating the length of time
# Get tweet JSON objects for all tweets in twitter archive data set
for er in error:
    try:
        tweet_data = api.get_status(tweet, tweet_mode = 'extended')
        # Get JSON Data of retweet and favorite count
        retweets = tweet_data['retweet_count']
        favorites = tweet_data['favorite_count']
        
        tweet_dic.append({'tweet_id': tweet,
                          'retweet': retweets,
                          'favorite': favorites})
        
    # Catch the exceptions of the TweepError
    except Exception:
        print(str(tweet)+ " _ " + str(e))
        second_error.append(er)
end = timer()
print(end - start)

5.6570282070024405


In [27]:
# write json object into text file
with open('tweet_json.txt', 'w') as outfile:
    json.dump(tweet_dic, outfile)

# Turn JSON file into a DataFrame
json_df = pd.read_json('tweet_json.txt')

In [28]:
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
favorite    2356 non-null int64
retweet     2356 non-null int64
tweet_id    2356 non-null int64
dtypes: int64(3)
memory usage: 55.3 KB


### Gathering Summary

The first step of data wrangling process is complete. 
- data is downloaded from exisiting file
- data is requested and downloaded from a Udacity server 
- data is query from titter's API
- All three data is gathered and imported into Jupyter Notebook


## Assess

The gathered data has to be assessed for the range of the cleaning issues required. When assessing, the type of cleaning is broken into two categories, tidy and dirty data issues. Below are each data issue that need to be address.

#### Json dataframe

In [29]:
# Check number of columns and entries
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
favorite    2356 non-null int64
retweet     2356 non-null int64
tweet_id    2356 non-null int64
dtypes: int64(3)
memory usage: 55.3 KB


In [30]:
# Visual inspection of data set
json_df

,favorite,retweet,tweet_id
0,38281,8394,892420643555336193
1,32809,6194,892177421306343426
2,24706,4099,891815181378084864
3,41603,8527,891689557279858688
4,39785,9241,891327558926688256
5,19961,3073,891087950875897856
6,11690,2040,890971913173991426
7,64618,18628,890729181411237888
8,27452,4214,890609185150312448
9,31497,7286,890240255349198849


In [31]:
# Check number of duplicate tweets
json_df.duplicated(subset= ['tweet_id','favorite','retweet']).sum()

14

#### Twitter archive dataframe

In [33]:
# Number of columns and duplicates
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [34]:
# Visual inspection of data set
twitter_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [85]:
# number of duplicate tweets
twitter_archive.duplicated(['tweet_id']).sum()

0

In [86]:
# Name columns shows incorrect dog names
twitter_archive.name

0        Phineas
1          Tilly
2         Archie
3          Darla
4       Franklin
5           None
6            Jax
7           None
8           Zoey
9         Cassie
10          Koda
11         Bruno
12          None
13           Ted
14        Stuart
15        Oliver
16           Jim
17          Zeke
18       Ralphus
19        Canela
20        Gerald
21       Jeffrey
22          such
23        Canela
24          None
25          None
26          Maya
27        Mingus
28         Derek
29        Roscoe
          ...   
2326       quite
2327           a
2328        None
2329        None
2330        None
2331        None
2332        None
2333          an
2334           a
2335          an
2336        None
2337        None
2338        None
2339        None
2340        None
2341        None
2342        None
2343        None
2344        None
2345         the
2346         the
2347           a
2348           a
2349          an
2350           a
2351        None
2352           a
2353          

### Image Prediction DataFrame

In [87]:
image_prediction

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [88]:
image_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [89]:
# Search for duplicates
image_prediction.duplicated('tweet_id').sum()

0

In [90]:
# Search for duplicates
image_prediction.duplicated(subset = ['jpg_url']).sum()

66

In [91]:
image_prediction.isna().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

### Quality
  ##### Twitter_archive


- tweet_id should be a string instead of an integer, timestamp column should be datetime data type, in_reply_to_status_id/user_id, retweeted_status_id/user_id should be srings instead of float, ratings_numerator/denominator, retweets, and favorites should be integers instead of floats

- Correct ratings scores 
- Correct dog descriptions
- 'Name' columns have incorrect names like 'a', 'such', 'None'
- rename columns 'text' = 'tweet_text', 'name' = 'dog_name', 'source' = 'source_application'
- Only interested in unique ratings, no retweets 
   
  ##### Image_prediction
- Tweet_id should be a string instead of an integer
- Some tweet_ids have the same jpg_url


  ##### json dataframe
- remove any duplicate tweet_id's


### Tidiness

- Merge all data tables together
- Dog description in columns instead of rows



### Cosmetic  Issues
- Drop unnecessary columns and NaN rows
- Retweeted_status_id/user_id should be dropped
- Too many columns in image prediciton could condense
- Source columncan be cleaned
- Missing values from images dataset only 2075 entires

Cleaning tidy data issues first is often the best way to start. When structral data issues are solved, cleaning quality data issues become simpler.

** Let's begin cleaning**

## Clean

In [405]:
# First step is to create a copy 
json_clean = json_df.copy()
twitter_clean = twitter_archive.copy()
image_clean = image_prediction.copy()

### Tidiness Data Cleaning

### Tidy Data Issue 1
 - Merge all data tables together

In [406]:
# The first step is going to merge all the data sets together.
twitter_master = pd.merge(twitter_clean, json_clean, how = 'left', on = 'tweet_id')
twitter_master = pd.merge(twitter_master, image_clean, how = 'left', on = 'tweet_id')

### Test Data Issue 1

In [407]:
# view twitter_master dataframe
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2370 entries, 0 to 2369
Data columns (total 30 columns):
tweet_id                      2370 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2370 non-null object
source                        2370 non-null object
text                          2370 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2311 non-null object
rating_numerator              2370 non-null int64
rating_denominator            2370 non-null int64
name                          2370 non-null object
doggo                         2370 non-null object
floofer                       2370 non-null object
pupper                        2370 non-null object
puppo                         2370 non-null object
favorite                      23

The large data table has many columns than can be condensed and will look good cosmetically. 

In [408]:
# Too many columns in image prediciton could condense
#dog prediction that are have labeled as True starting with the first prediction
#and move down to second and third predictions are kept and put into 'predicted_dog' column. 
#The algorithm's confidence is stored in the 'confidence_algore' column.
predicted_dog = []
confidence_algore = []
def something(data):
    if data['p1_dog'] == True:
        predicted_dog.append(data['p1'])
        confidence_algore.append(data['p1_conf'])
    elif data['p2_dog'] == True:
        predicted_dog.append(data['p2'])
        confidence_algore.append(data['p2_conf'])
    elif data['p3_dog'] == True:
        predicted_dog.append(data['p3'])
        confidence_algore.append(data['p3_conf'])
    else:
        predicted_dog.append('NaN')
        confidence_algore.append(0)
# Too many columns in image prediciton could condense
twitter_master.loc[:, 'p1': 'p3_dog'].apply(something, axis = 1)
twitter_master['predicted_dog'] = np.array(predicted_dog)
twitter_master['confidence_algore'] = np.array(confidence_algore)
# Too many columns in image prediciton could condense
twitter_master.drop(columns = twitter_master.loc[:,'img_num':'p3_dog'].columns, inplace = True)

Source column can be cleaned to only indicate source instead of entire link

In [409]:
# Clean the source column
twitter_master['source'] = twitter_master['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])


In [410]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2370 entries, 0 to 2369
Data columns (total 22 columns):
tweet_id                      2370 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2370 non-null object
source                        2370 non-null object
text                          2370 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2311 non-null object
rating_numerator              2370 non-null int64
rating_denominator            2370 non-null int64
name                          2370 non-null object
doggo                         2370 non-null object
floofer                       2370 non-null object
pupper                        2370 non-null object
puppo                         2370 non-null object
favorite                      23

### Tidy Data Issue 2 & 3

- Only interested in unique ratings, no retweets
- Drop duplicate tweet_ids


In [411]:
# Row entries that have retweets need to be dropped
twitter_master = twitter_master[pd.isnull(twitter_master['retweeted_status_id'])]

# Drop duplicate tweet_id total of 14 rows
twitter_master.drop_duplicates(inplace = True)

# Drop any retweet related columns
twitter_master.drop(columns = ['retweeted_status_id', 'retweeted_status_user_id',
                                'retweeted_status_timestamp'], inplace = True)

In [412]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null object
source                   2175 non-null object
text                     2175 non-null object
expanded_urls            2117 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     2175 non-null object
doggo                    2175 non-null object
floofer                  2175 non-null object
pupper                   2175 non-null object
puppo                    2175 non-null object
favorite                 2174 non-null float64
retweet                  2174 non-null float64
jpg_url                  1994 non-null object
predicted_dog            2175 non-null object
confidence_algore        2175 non-null float64
dtypes: float64(5), i

In [413]:
# Dropping NaN rows
twitter_master.dropna(subset = ['jpg_url', 'retweet', 'favorite'], inplace = True)

### Test Data Issue 2

In [414]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 2355
Data columns (total 19 columns):
tweet_id                 1993 non-null int64
in_reply_to_status_id    23 non-null float64
in_reply_to_user_id      23 non-null float64
timestamp                1993 non-null object
source                   1993 non-null object
text                     1993 non-null object
expanded_urls            1993 non-null object
rating_numerator         1993 non-null int64
rating_denominator       1993 non-null int64
name                     1993 non-null object
doggo                    1993 non-null object
floofer                  1993 non-null object
pupper                   1993 non-null object
puppo                    1993 non-null object
favorite                 1993 non-null float64
retweet                  1993 non-null float64
jpg_url                  1993 non-null object
predicted_dog            1993 non-null object
confidence_algore        1993 non-null float64
dtypes: float64(5), i

### Tidy Data Issue 4

- The four dog description each have there own columns when they should be condensed into a row using the melt function

In [415]:
# index of multiple dog descriptions
tweetid = twitter_master.loc[(twitter_master[['doggo', 'floofer', 'pupper', 'puppo']] != 'None').sum(axis = 1)> 1].tweet_id.tolist()


In [416]:
# Create columns list for the melt function
columns_condensed = ['doggo', 'floofer', 'pupper', 'puppo']
columns_unchanged = []
for x in twitter_master.columns:
    if x not in columns_condensed:
        columns_unchanged.append(x)

In [417]:
# Melt the columns
twitter_master = pd.melt(twitter_master, id_vars = columns_unchanged, value_vars = columns_condensed,
       var_name = 'column_names', value_name = 'dog_description')


In [418]:
# convert into string to locate easier 
tweetid = [str(x) for x in tweetid]
# Convert rows with multiple dog descriptions to 'multiple'
twitter_master.loc[twitter_master['tweet_id'].isin(tweetid), 'dog_description'] = 'multiple'

In [419]:
# Sort the dog description values and then keep the last duplicates in the table 
twitter_master = twitter_master.sort_values('dog_description').drop_duplicates(
    subset = ['tweet_id'], keep = 'last')

In [420]:
# drop 'column_names' column and then reset index
twitter_master.drop(columns = ['column_names'], inplace = True)
twitter_master.reset_index(drop = True, inplace = True)

### Test Data Issue 4

In [421]:
twitter_master.dog_description.value_counts()

None        1687
pupper       203
doggo         63
puppo         22
multiple      11
floofer        7
Name: dog_description, dtype: int64

All the columns for the dog description are in one column. The majority of entries for dog description are 'None'. 

## Quality Data Cleaning

### Quality Data Issue 1
No operations will be done on 'tweet_id' ,'in_reply_to_status_id', 'in_reply_to_user_id' columns and should be converted into strings. The 'favorite' and 'retweets' columns should be converted into integers. The timestamp column are easier to manipulate as datetime data types.

- Convert Columns to the appropriate data types

In [422]:
# Change columns from integers/floats to strings
twitter_master = twitter_master.astype({'tweet_id': 'str', 'in_reply_to_status_id': 'str',
                                       'in_reply_to_user_id': 'str'})
# Change columns from floats to integers
twitter_master = twitter_master.astype({'favorite': 'int32', 'retweet':'int32'})
# Change date to datetime
twitter_master['timestamp'] = pd.to_datetime(twitter_master['timestamp'])

### Test Data Issue 1

In [423]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 16 columns):
tweet_id                 1993 non-null object
in_reply_to_status_id    1993 non-null object
in_reply_to_user_id      1993 non-null object
timestamp                1993 non-null datetime64[ns]
source                   1993 non-null object
text                     1993 non-null object
expanded_urls            1993 non-null object
rating_numerator         1993 non-null int64
rating_denominator       1993 non-null int64
name                     1993 non-null object
favorite                 1993 non-null int32
retweet                  1993 non-null int32
jpg_url                  1993 non-null object
predicted_dog            1993 non-null object
confidence_algore        1993 non-null float64
dog_description          1993 non-null object
dtypes: datetime64[ns](1), float64(1), int32(2), int64(2), object(10)
memory usage: 233.6+ KB


### Quality Data Issue 2

There are certain entries under 'name' columns that are incorrect dog names. Certain dog names are labeled as either 'None', 'is', 'O', 'Such', etc., even when the dog name is presented in the text portion of the tweet. 
- Change dog names that are incorrect

In [424]:
# Create a function that evaluates the fist letter of the name as lower case or 'None'
def islower(x):
    if x[0].islower() or x == 'None':
        return True
    else:
        return False


In [425]:
# Created a different table with dog names that potentially need to be corrected
fix_dogname = twitter_master[twitter_master.name.apply(islower)]

There are 644 rows that have either 'None' or a name starting with a lower case letter. What are all the names? 

In [426]:
# Get a value count for all the different names
twitter_master[twitter_master.name.apply(islower)].name.value_counts()

None            546
a                55
the               7
an                6
one               4
very              4
just              3
quite             3
getting           2
incredibly        1
this              1
unacceptable      1
his               1
space             1
officially        1
not               1
actually          1
my                1
all               1
infuriating       1
by                1
such              1
light             1
Name: name, dtype: int64

There are no entries that have any potential dog names in the new dataframe. Using regex, any potential dog names can be extracted into a new data table and then merged back into the original dataframe.

In [427]:
# Create a dataframe with all the corrections
correction = fix_dogname['text'].str.extract(r'that\s([D-Z]\w+)')
correction['second'] = fix_dogname['text'].str.extract(r'name| named\s(\w*\s*[A-Z]\w+)', expand = False)
correction['third'] = fix_dogname['text'].str.extract(r'\whis\sis\s([A-Z]\w+)', expand = False)
correction.rename( columns = {0 : 'first'} , inplace = True)



In [428]:
# Drop rows that do not have any of correction in it and turn NaN values into empty strings
correction.dropna(how = 'all', inplace = True)
correction.replace(np.nan, '', inplace = True)
# Create a final column combining all the corrections from the three columns
correction['name'] = correction['first'] + correction['second'] + correction['third']
correction.head(3)

,first,second,third,name
4,,Kohl,,Kohl
8,,Tickles,,Tickles
13,,Klint,,Klint


In [429]:
# any entry that is lowercase becomes 'None'
fix_dogname.loc[:, 'name'] = 'None'
fix_dogname.name.value_counts()

/Users/Erudition/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


None    644
Name: name, dtype: int64

In [430]:
# update the twitter_master dataframe with correction['final'] column
fix_dogname.update(correction['name'])
fix_dogname.name.value_counts()

/Users/Erudition/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:5082: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[col] = expressions.where(mask, this, that)


None         615
Chuk           1
Octaviath      1
Cherokee       1
Johm           1
Jake           1
Berta          1
Alphred        1
Rufus          1
Klint          1
Leroi          1
Zeus           1
Alfredo        1
Jessiga        1
Guss           1
Bretagne       1
Kohl           1
Spork          1
Cheryl         1
Wylie          1
Alfonso        1
Hemry          1
Tickles        1
Kip            1
Pippa          1
Jacob          1
Pepe           1
Toby           1
Yoshi          1
Blue           1
Name: name, dtype: int64

In [431]:
# update the twitter_master dataframe with correction['final'] column
twitter_master.update(fix_dogname)

There are certain entries that do have multiple dog names that were identified in the Tidy Data Issue 4 portion of this report. For the sake of uniformity and time, the dog names should be changed to 'multiple dogs'

In [432]:
twitter_master

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,favorite,retweet,jpg_url,predicted_dog,confidence_algore,dog_description
0,667437278097252352,nan,nan,2015-11-19 20:20:22,Twitter for iPhone,Never seen this breed before. Very pointy pup....,https://twitter.com/dog_rates/status/667437278...,10.0,10.0,None,453.0,243.0,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,NaN,0.000000,None
1,667443425659232256,nan,nan,2015-11-19 20:44:47,Twitter for iPhone,Exotic dog here. Long neck. Weird paws. Obsess...,https://twitter.com/dog_rates/status/667443425...,6.0,10.0,None,797.0,592.0,https://pbs.twimg.com/media/CUM8QZwW4AAVsBl.jpg,NaN,0.000000,None
2,667453023279554560,nan,nan,2015-11-19 21:22:56,Twitter Web Client,Meet Cupcake. I would do unspeakable things fo...,https://twitter.com/dog_rates/status/667453023...,11.0,10.0,Cupcake,319.0,91.0,https://pbs.twimg.com/media/CUNE_OSUwAAdHhX.jpg,Labrador_retriever,0.825670,None
3,667455448082227200,nan,nan,2015-11-19 21:32:34,Twitter Web Client,This is Reese and Twips. Reese protects Twips....,https://twitter.com/dog_rates/status/667455448...,7.0,10.0,Reese,194.0,60.0,https://pbs.twimg.com/media/CUNHMXTU8AAS3HH.jpg,Tibetan_terrier,0.676376,None
4,667470559035432960,nan,nan,2015-11-19 22:32:36,Twitter Web Client,This is a northern Wahoo named Kohl. He runs t...,https://twitter.com/dog_rates/status/667470559...,11.0,10.0,Kohl,259.0,100.0,https://pbs.twimg.com/media/CUNU78YWEAECmpB.jpg,toy_poodle,0.304175,None
5,667491009379606528,nan,nan,2015-11-19 23:53:52,Twitter Web Client,Two dogs in this one. Both are rare Jujitsu Py...,https://twitter.com/dog_rates/status/667491009...,7.0,10.0,None,540.0,232.0,https://pbs.twimg.com/media/CUNniSlUYAEj1Jl.jpg,borzoi,0.852088,None
6,667495797102141441,nan,nan,2015-11-20 00:12:54,Twitter Web Client,This is Philippe from Soviet Russia. Commandin...,https://twitter.com/dog_rates/status/667495797...,9.0,10.0,Philippe,532.0,280.0,https://pbs.twimg.com/media/CUNr4-7UwAAg2lq.jpg,Chihuahua,0.143957,None
7,667502640335572993,nan,nan,2015-11-20 00:40:05,Twitter Web Client,Say hello to Hall and Oates. Oates is winking ...,https://twitter.com/dog_rates/status/667502640...,11.0,10.0,Hall,539.0,224.0,https://pbs.twimg.com/media/CUNyHTMUYAAQVch.jpg,Labrador_retriever,0.996709,None
8,667509364010450944,nan,nan,2015-11-20 01:06:48,Twitter Web Client,This a Norwegian Pewterschmidt named Tickles. ...,https://twitter.com/dog_rates/status/667509364...,12.0,10.0,Tickles,6876.0,2175.0,https://pbs.twimg.com/media/CUN4Or5UAAAa5K4.jpg,beagle,0.636169,None
9,667517642048163840,nan,nan,2015-11-20 01:39:42,Twitter Web Client,This is Dook &amp; Milo. Dook is struggling to...,https://twitter.com/dog_rates/status/667517642...,8.0,10.0,Dook,374.0,196.0,https://pbs.twimg.com/media/CUN_wiBUkAAakT0.jpg,Italian_greyhound,0.125176,None


### Test Data Issue 2

In [433]:
# All the names in the dataset
twitter_master.name.value_counts().index.tolist()

['None',
 'Oliver',
 'Cooper',
 'Lucy',
 'Charlie',
 'Tucker',
 'Penny',
 'Winston',
 'Toby',
 'Sadie',
 'Lola',
 'Daisy',
 'Jax',
 'Stanley',
 'Bo',
 'Koda',
 'Bella',
 'Buddy',
 'Oscar',
 'Leo',
 'Dave',
 'Chester',
 'Louis',
 'Bailey',
 'Rusty',
 'Milo',
 'Scout',
 'Reggie',
 'Jerry',
 'Sophie',
 'Bentley',
 'Clark',
 'Alfie',
 'Duke',
 'Jack',
 'Derek',
 'Gary',
 'Finn',
 'Clarence',
 'Cassie',
 'Larry',
 'Phil',
 'Brody',
 'Winnie',
 'Dexter',
 'Bear',
 'Gus',
 'Oakley',
 'Scooter',
 'George',
 'Maggie',
 'Archie',
 'Chip',
 'Olive',
 'Wyatt',
 'Shadow',
 'Sunny',
 'Vincent',
 'Samson',
 'Riley',
 'Gizmo',
 'Rufus',
 'Max',
 'Lily',
 'Pippa',
 'Walter',
 'Rosie',
 'Boomer',
 'Sebastian',
 'Ellie',
 'Zeke',
 'Jeffrey',
 'Wallace',
 'Mia',
 'Moose',
 'Louie',
 'Steven',
 'Earl',
 'Ruby',
 'Calvin',
 'Otis',
 'Wilson',
 'Malcolm',
 'Beau',
 'Waffles',
 'Bruce',
 'Hank',
 'Loki',
 'Reese',
 'Nala',
 'Jimothy',
 'Sammy',
 'Kyle',
 'Gerald',
 'Zoey',
 'Lincoln',
 'Chipson',
 'Jeph',
 'O

### Quality Data Issue 4 

The in_repy_to_status_id and in_reply_to_user_id columns are not necessary for any analysis and can be dropped.

In [434]:
twitter_master.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], 1, inplace = True)

### Test Data Issue 4

In [435]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 14 columns):
tweet_id              1993 non-null object
timestamp             1993 non-null datetime64[ns]
source                1993 non-null object
text                  1993 non-null object
expanded_urls         1993 non-null object
rating_numerator      1993 non-null float64
rating_denominator    1993 non-null float64
name                  1993 non-null object
favorite              1993 non-null float64
retweet               1993 non-null float64
jpg_url               1993 non-null object
predicted_dog         1993 non-null object
confidence_algore     1993 non-null float64
dog_description       1993 non-null object
dtypes: datetime64[ns](1), float64(5), object(8)
memory usage: 218.1+ KB


###  Quality Data Issue 5

Rename columns for better description

In [436]:
twitter_master.rename(columns = {'timestamp': 'tweet_date', 'source': 'tweet_source', 'text': 'tweet_text', 
                                        'expanded_urls': 'tweet_url','name': 'dog_name', 'jpg_url': 'tweet_picture', 
                                        'favorite': 'tweet_favorites', 'retweet': 'tweet_retweets',
                                        'prediction_algorithm' : 'dog_breed', 'confidence_algore': 'algorithm_confidence'}, inplace = True)

### Test Data Issue 5

In [437]:
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 14 columns):
tweet_id                1993 non-null object
tweet_date              1993 non-null datetime64[ns]
tweet_source            1993 non-null object
tweet_text              1993 non-null object
tweet_url               1993 non-null object
rating_numerator        1993 non-null float64
rating_denominator      1993 non-null float64
dog_name                1993 non-null object
tweet_favorites         1993 non-null float64
tweet_retweets          1993 non-null float64
tweet_picture           1993 non-null object
predicted_dog           1993 non-null object
algorithm_confidence    1993 non-null float64
dog_description         1993 non-null object
dtypes: datetime64[ns](1), float64(5), object(8)
memory usage: 218.1+ KB


###  Quality Data Issue 6

Correct ratings

Narrow the dataframe to incorporate denominator values that are not 10.0.

In [438]:
twitter_master.loc[twitter_master['rating_denominator'] != 10].rating_denominator.value_counts()

50.0     3
80.0     2
11.0     2
170.0    1
150.0    1
70.0     1
20.0     1
7.0      1
120.0    1
40.0     1
90.0     1
130.0    1
110.0    1
2.0      1
Name: rating_denominator, dtype: int64

After comparing the twitter_text with each rating_denominator value, the values that were a mismatch were manually corrected.

In [442]:
# Manual correction of each rating denominator incorrectly labled
twitter_master.loc[twitter_master['rating_denominator']== 2.0, 'rating_denominator'] = 10
twitter_master.loc[twitter_master['rating_denominator']== 11.0, 'rating_denominator'] = 10
twitter_master.loc[twitter_master['rating_denominator']== 7.0, 'rating_denominator'] = np.nan
twitter_master.loc[twitter_master['dog_name']== 'Bluebert', 'rating_denominator'] = 10

In [443]:
twitter_master.loc[twitter_master['rating_denominator'] != 10].rating_denominator.value_counts()

50.0     2
80.0     2
None     1
110.0    1
170.0    1
90.0     1
150.0    1
40.0     1
120.0    1
70.0     1
20.0     1
130.0    1
Name: rating_denominator, dtype: int64

## Storing Data

In [475]:
twitter_master.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')
twitter_master = pd.read_csv('twitter_archive_master.csv')
twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1993 entries, 0 to 1992
Data columns (total 14 columns):
tweet_id                1993 non-null int64
tweet_date              1993 non-null object
tweet_source            1993 non-null object
tweet_text              1993 non-null object
tweet_url               1993 non-null object
rating_numerator        1993 non-null float64
rating_denominator      1993 non-null object
dog_name                1993 non-null object
tweet_favorites         1993 non-null float64
tweet_retweets          1993 non-null float64
tweet_picture           1993 non-null object
predicted_dog           1685 non-null object
algorithm_confidence    1993 non-null float64
dog_description         1993 non-null object
dtypes: float64(4), int64(1), object(9)
memory usage: 218.1+ KB
